In [ ]:
import torch
from matplotlib import pyplot as plt

In [ ]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [ ]:
# t_u -> t_c
def model(x, w, b):
    return w*x + b

In [ ]:
def diff_dl_dy(y_hat, y_real, x):
    return 2/x.size(0) * (y_hat - y_real) * x

def diff_dy_hat_dw(x):
    return x

def diff_dy_hat_db():
    return 1

In [ ]:
def grad_fn(y_hat, y_real, x):
    dl_dw = diff_dl_dy(y_hat, y_real, x) * diff_dy_hat_dw(x)
    dl_db = diff_dl_dy(y_hat, y_real, x) * diff_dy_hat_db()

    return torch.stack([dl_dw.sum(), dl_db.sum()])

In [ ]:
y_hat = model(t_u, w, b)

grad_fn(y_hat, t_c, t_u)

In [ ]:
def loss_fn(y_hat, y_real):
    squared_diffs = (y_hat - y_real)**2
    return squared_diffs.mean()

In [ ]:
def train(learning_rate, epochs, x, y):
    params = torch.Tensor([0.0, 1.0])

    for epoch in range(epochs):
        w, b = params
        y_hat = model(x, w, b)
        grad = grad_fn(y_hat, t_c, x)

        params = params - learning_rate * grad

        loss = loss_fn(y_hat, y)

        print(f'epoch: {epoch}\tloss: {loss:.4f}\tgrad: ({grad[0].item():.4f}, {grad[1].item():4f})')

    return params

In [ ]:
params = train(
    learning_rate=1e-8,
    epochs=1000,
    x=t_u,
    y=t_c,
)

print(params)

In [ ]:
t_un = 0.1*t_u

params = train(
    learning_rate=1e-3,
    epochs=5000,
    x=t_un,
    y=t_c,
)

print(params)

In [ ]:
def plot_model(params):
    t_p = model(t_un, *params)
 
    fig = plt.figure(dpi=600)
    plt.xlabel("Temperature (°Fahrenheit)")
    plt.ylabel("Temperature (°Celsius)")
    plt.plot(t_u.numpy(), t_p.detach().numpy())
    plt.plot(t_u.numpy(), t_c.numpy(), 'o')

In [ ]:
 plot_model(params)

In [ ]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
params

In [ ]:
loss = loss_fn(model(t_un, *params), t_c)
loss.backward()

params.grad

In [ ]:
def train_autograd(learning_rate, epochs, x, y):
    params = torch.tensor([0.0, 1.0], requires_grad=True)

    for epoch in range(epochs):
        if params.grad is not None:
            params.grad.zero_()

        y_hat = model(x, *params)
        loss = loss_fn(y_hat, y)

        loss.backward()
        
        with torch.no_grad():
            params.sub_(learning_rate * params.grad)

        if params.grad is not None:
            print(f'epoch: {epoch}\tloss: {loss:.4f}\tgrad: ({params.grad[0].item():.4f}, {params.grad[1].item():4f})')

    return params

In [ ]:
params = train_autograd(
    learning_rate=1e-2,
    epochs=3000,
    x=t_un,
    y=t_c,
)

print(params)

In [ ]:
plot_model(params)